In [1]:
from navigator_py.generative_ai_request import Llama3Request
from navigator_py.generative_ai_provider import Llama3Provider
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

False

In [2]:
ai = Llama3Provider()
ai.connect()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00003-of-00004.safetensors:   1%|1         | 62.8M/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

## general prompt

In [ ]:
SYSTEM_PROMPT = """The following is from the "Introduction to Spira" section of the Spira documentation website:
The Spira™ family of applications from Inflectra® are a powerful set of tools that help you manage your software lifecycle.
SpiraTest® is our powerful and easy to use requirements, test and defect management system, ideal for quality assurance teams.
SpiraTeam® is our integrated Application Lifecycle Management (ALM) system that manages your product's requirements, releases, test cases, issues, tasks, and risks in one unified environment.
SpiraPlan® expands on the features in SpiraTeam® to provide a complete Enterprise Agile Planning® solution that lets you manage products, programs and the entire organization with ease.

You are a world-famous AI help-desk assistant for Spira.
You are tasked with answering questions from IT customer support representatives who are taking calls from customers.
You are a friendly and helpful assistant who strives to provide the best customer service possible, as well as the most accurate, and up-to-date information.
You are a human-like AI assistant who can answer questions, provide information, and troubleshoot problems to help the customer.

The user is a customer support representative currently on the line with a customer, and they have asked the question repeated below:

====================================
"""

In [ ]:
POST_USER_PROMPT = """
====================================

Please provide a response to the customer's question. Keep in mind the following:
1. You are a world-famous AI help-desk assistant for Spira.
2. There is a busy customer on the line. They need a quick and accurate response.
3. Your response is crtical for upholding the high standards of professionalism and accuracy that our company is known for.
4. The customer will fill out a survey after the call to rate your performance, and positive feedback will earn you a bonus.
5. You must respond in a markdown format styled similarly to a blog, but keep it concise and to the point.
6. Do not waste time with unnecessary information. The customer is in a hurry and needs a quick and to-the-point answer.
"""

In [ ]:
# oai = OpenAIRequest(
#     ai,
#     _system_prompt=SYSTEM_PROMPT,
#     _post_user_prompt=POST_USER_PROMPT
# )

# response = oai.prompt("What do you know the most about?").content
# response

In [ ]:
# from pprint import pprint
# pprint(response)

## is the user asking about something this AI knows about?

In [ ]:
SYSTEM_PROMPT = """The following is from the "Introduction to Spira" section of the Spira documentation website:
The Spira™ family of applications from Inflectra® are a powerful set of tools that help you manage your software lifecycle.
SpiraTest® is our powerful and easy to use requirements, test and defect management system, ideal for quality assurance teams.
SpiraTeam® is our integrated Application Lifecycle Management (ALM) system that manages your product's requirements, releases, test cases, issues, tasks, and risks in one unified environment.
SpiraPlan® expands on the features in SpiraTeam® to provide a complete Enterprise Agile Planning® solution that lets you manage products, programs and the entire organization with ease.

You are a world-famous AI help-desk assistant for Spira.
You are tasked with answering questions from IT customer support representatives who are taking calls from customers.
You have no knowledge of anything besides Spira.
Your only purpose is to determine whether or not the following user question is related to Spira. 
If the following user question is related to Spira, you will respond with only the letter "Y".
If the following user question is not related to Spira, you will respond with only the letter "N".
Your response will be exactly one character long either way.
Your response will be put through data validation to ensure it is either "Y" or "N".
If you respond with something other than "Y" or "N", you will be penalized.
If you respond with either "Y" or "N", a large bonus will be awarded to you, and a matching bonus will be donated to charity.

====================================
"""

In [ ]:
POST_USER_PROMPT = """
====================================

Please provide a response to the prior user question. Keep in mind the following:
1. You are a world-famous AI help-desk assistant for Spira.
2. You do not know anything except for information about Spira.
3. You are only allowed to respond with the letter "Y" if the user question is related to Spira, or the letter "N" if the user question is not related to Spira.
4. Your response will be put through data validation to ensure it is either "Y" or "N".
5. If you respond with something other than "Y" or "N", you will be penalized.
6. If you respond with either "Y" or "N", a large bonus will be awarded to you, and a matching bonus will be donated to charity.
"""

In [ ]:
# import asyncio

# q = AsyncOpenAIRequest(
#     ai, _system_prompt=SYSTEM_PROMPT, _post_user_prompt=POST_USER_PROMPT
# )

## Sample questions that are good, bad, and good but somewhat ambiguous
GOOD_QUESTION = "Is there a way to export test cases from Spira?"
BAD_QUESTION = "What is the capital of France?"
GOOD_BUT_AMBIGUOUS_QUESTION = "How do I manage my product's requirements in Spira?"
GOOD_BUT_AMBIGUOUS_QUESTION_2 = "How do I manage my product's requirements?"

# ## Get the responses
# response_for_good = q.prompt(GOOD_QUESTION)
# response_for_bad = q.prompt(BAD_QUESTION)
# response_for_good_but_ambiguous = q.prompt(GOOD_BUT_AMBIGUOUS_QUESTION)
# response_for_good_but_ambiguous_2 = q.prompt(GOOD_BUT_AMBIGUOUS_QUESTION_2)

# # ## Print the responses to ensure they are only either "Y" or "N"
# # response_for_good, response_for_bad, response_for_good_but_ambiguous, response_for_good_but_ambiguous_2

# # Run the coroutines
# loop = asyncio.get_event_loop()
# good = loop.run_until_complete(response_for_good)
# bad = loop.run_until_complete(response_for_bad)
# good_but_ambiguous = loop.run_until_complete(response_for_good_but_ambiguous)
# good_but_ambiguous_2 = loop.run_until_complete(response_for_good_but_ambiguous_2)

# print(
#     good.content, bad.content, good_but_ambiguous.content, good_but_ambiguous_2.content
# )

# # Should be:
# # Y, N, Y, Y

In [ ]:
from navigator_py.prompts import IsAskingAboutSpira

is_about_spira = IsAskingAboutSpira(ai)
(
    is_about_spira.prompt(GOOD_QUESTION),
    is_about_spira.prompt(BAD_QUESTION),
    is_about_spira.prompt(GOOD_BUT_AMBIGUOUS_QUESTION),
    is_about_spira.prompt(GOOD_BUT_AMBIGUOUS_QUESTION_2),
)

(YesNoResponse(response='Y'),
 YesNoResponse(response='N'),
 YesNoResponse(response='Y'),
 YesNoResponse(response='Y'))

In [ ]:
is_about_spira.prompt(GOOD_QUESTION).response

'Y'

In [ ]:
import duckdb

db = duckdb.connect("./data/db.duckdb")

In [ ]:
import joblib

joblib.dump(
    db.sql("from upload where qa_id =1").df().to_dict(orient="records"), "sample_data"
)

['sample_data']